# The openLCA Result API

openLCA 2 provides a harmonized result interface that can be called from
different APIs (Java, JSON-RPC, gRPC, Rest). All these APIs call the same
backend which is implemented in the openLCA kernel. This kernel is independent
from the openLCA user interface and can be integrated as a service into other
applications.

The idea of the result interface is not to provide some ready-to-use charts and
tables but to provide all possible building blocks with which such higher level
result views can be created (charts, tables, upstream trees, Sankey diagrams).
Thus, the result interface has many methods that often look quite similar but
they have their purpose for efficiently creating higher level result views. This
notebook tries to explain what these methods do with examples using the [openLCA
Python IPC API v2](https://github.com/GreenDelta/olca-ipc.py). The calculation
is sometimes explained using [standard LCA matrix algebra](.#) but this does mean
that the implementation uses exactly these formulas when calculating results.

## Setup

In order to run the examples in this document, you need to have the olca-ipc.py
module installed and an IPC server in openLCA runnig (`Tools > Developer tools >
IPC Server`). First, make sure that the `olca-ipc` and `olca-schema` modules are
installed. We also use `pandas` and `matplotlib` for visualisations:

In [ ]:
!pip list | grep 'olca\|pandas'

Now, we import the required modules:

In [ ]:
import olca_ipc as ipc
import olca_schema as lca
import olca_schema.results as res
import pandas as pd

## Calculation and result state

### `/calculate`

In order to run a calculation, we first need to create a calculation setup. In
this setup, we define the calculation target (a process or product system) and
configure options like the LCIA method that should be used, if life cycle costs
should be calculated or not, which parameters should be redefined etc. With this
setup we can start a calculation:

In [ ]:
setup = res.CalculationSetup(
    target=lca.Ref(
        model_type="ProductSystem", id="7c328e9b-d8e3-402b-a1ac-95620d021b99"
    ),
    impact_method=lca.Ref(
        model_type="ImpactMethod", id="787c02f1-d1f2-36d6-8e06-2307cc3ebebc"
    ),
)
client = ipc.Client(8080)
result = client.calculate(setup)

### `/state`

Starting the calculation directly returns a result object but this result may
isn't ready yet. This is because when we start a calculation, it is first put in
a calculation queue and there may are other calculations that are still running
until our calculation is scheduled and then, we may have to wait a bit depending
how big the system is we calculate. We can inspect the result state and could
actively wait unit the `is_ready` attribute is `True`. The result can also be an
error so we should also check this:

In [ ]:
result.get_state()

We can also just use the convenience method `wait_until_ready`:

In [ ]:
result.wait_until_ready()

### `/dispose`
We will call the method `dispose` at the end of this notebook but mention it
already here: a calculated result can allocate quite some resources in the
openLCA backend depending on the calculated system. If we do not need the result
anymore, we should dispose the result to free these resources especially, if we
want to run multiple calculations.

In [ ]:
# we do this later
# result.dispose()

In [ ]:
tech_flows = result.get_tech_flows()
pd.DataFrame(
  [(tf.provider.name, tf.flow.name) for tf in tech_flows],
  columns=["Provider", "Flow"]
).head()

In [ ]:
envi_flows = result.get_envi_flows()
pd.DataFrame(
  [(ef.is_input, ef.flow.name, ef.flow.category) for ef in envi_flows],
  columns=["Is input?", "Flow", "Category"]
).head()

In [ ]:
impact_categories = result.get_impact_categories()
pd.DataFrame(
    [i.name for i in impact_categories],
    columns=["Impact category"]
)

## Technosphere flows

### `/total-requirements`

This method returns the total requirements $t$ of technosphere flows to
fulfill the demand of a product system. The amounts are given in the respective
reference units of the technosphere flows. $t$ can be calculated by scaling the
diagonal of the technosphere matrix $A$ with the scaling vector $s$:

$$
t = diag(s) * diag(A)
$$

In [ ]:
t = result.get_total_requirements()
pd.DataFrame(
  [(v.tech_flow.flow.name, v.amount, v.tech_flow.flow.ref_unit) for v in t],
  columns=["Tech. flow", "Amount", "Unit"]
).head()